In [1]:
%load_ext autoreload
%autoreload 2

In [2]:




import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader 
from abc import ABC
from tqdm import tqdm
import torchvision
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random




config = {}
config["seed"] = 42
seed = config["seed"]
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)




config["num_clients_per_cluster"] = 8
config["num_clusters"] = 4
config["num_clients"] = config["num_clients_per_cluster"]*config["num_clusters"]
config["dataset"] = "mnist"
DATASET_LIB = {"mnist" : torchvision.datasets.MNIST, "emnist": torchvision.datasets.EMNIST, "cifar10": torchvision.datasets.CIFAR10}
config["dataset_dir"] = "./experiments/dataset"
config["results_dir"] = "./experiments/results"
config["results_dir"] = os.path.join(config["results_dir"], config["dataset"], "seed_{}".format(seed))

os.makedirs(config["results_dir"], exist_ok=True)

def split(dataset_size, num_clients, shuffle):
    split_idx = []
    all_idx = np.arange(dataset_size)
    if shuffle:
        all_idx = np.random.permutation(all_idx)
    split_idx = np.array_split(all_idx, num_clients)
    return split_idx

def dataset_split(train_data, test_data, num_clients, shuffle):
    train_size = train_data[0].shape[0]
    train_split_idx = split(train_size, num_clients, shuffle)
    train_chunks = [(train_data[0][train_split_idx[client]], train_data[1][train_split_idx[client]]) for client in range(num_clients)]
    test_size = test_data[0].shape[0]
    test_split_idx = split(test_size, num_clients, shuffle)
    test_chunks = [(test_data[0][test_split_idx[client]], test_data[1][test_split_idx[client]]) for client in range(num_clients)]
    return train_chunks, test_chunks

def make_client_datasets(config, num_clusters = 10):
    train_chunks_total = []
    test_chunks_total = []
    train_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=True)
    test_dataset = DATASET_LIB[config["dataset"]](root = config['dataset_dir'], download = True, train=False)

    train_data = (train_dataset.data, train_dataset.targets)
    test_data = (test_dataset.data, test_dataset.targets)
    for i in range(config["num_clusters"]):
        train_chunks, test_chunks = dataset_split(train_data, test_data, config["num_clients_per_cluster"], shuffle=True)
        train_chunks_total += train_chunks
        test_chunks_total += test_chunks
    return train_chunks_total, test_chunks_total


class ClientDataset(Dataset):
    def __init__(self, data,transforms = None):
        super(ClientDataset,self).__init__()
        self.data = data[0]
        self.labels = data[1]
        self.transforms = transforms

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        idx_data = self.data[idx]
        if self.transforms is not None:
            transformed_data =self.transforms(idx_data)
        else:
            transformed_data = idx_data
        idx_labels = self.labels[idx]
        return (transformed_data.unsqueeze(0).float(), idx_labels)



In [3]:
class Client():
    def __init__(self, train_data, test_data, client_id,  train_transforms, test_transforms, train_batch_size, test_batch_size, save_dir):
        self.trainset = ClientDataset(train_data, train_transforms)
        self.testset = ClientDataset(test_data, test_transforms)
        self.trainloader = DataLoader(self.trainset, batch_size = train_batch_size, shuffle=True, num_workers=4)
        self.testloader = DataLoader(self.testset, batch_size = test_batch_size, shuffle=False, num_workers=4)
        self.train_iterator = iter(self.trainloader)
        self.test_iterator = iter(self.testloader)
        self.client_id = client_id
        self.save_dir = os.path.join(save_dir, "init", "client_{}".format(client_id))

    def sample_batch(self, train=True):
        iterator = self.train_iterator if train else self.test_iterator
        try:
            (data, labels) = next(iterator)
        except StopIteration:
            if train:
                self.train_iterator = iter(self.trainloader)
                iterator = self.train_iterator
            else:
                self.test_iterator = iter(self.testloader)
                iterator = self.test_iterator
            (data, labels) = next(iterator)
        return (data, labels)


In [4]:
train_chunks, test_chunks = make_client_datasets(config)


In [5]:
import torchvision.transforms.functional as TF


class RotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angle):
        self.angle = angle

    def __call__(self, x):
        return TF.rotate(x, self.angle)


config["train_batch"] = 512
config["test_batch"] = 512
client_loaders = []
for i in range(config["num_clusters"]):
    for j in range(config["num_clients_per_cluster"]):
        idx = i * config["num_clients_per_cluster"] + j
        x_train = train_chunks[idx][0]
        x_test = test_chunks[idx][0]
        if i >0:
            x_train = torch.rot90(x_train, i, [1, 2])
            x_test = torch.rot90(x_test, i, [1, 2])
        client_loaders.append(
            Client(
                (x_train, train_chunks[idx][1]),
                (x_test, test_chunks[idx][1]),
                idx,
                train_transforms=None,
                test_transforms=None,
                train_batch_size=config["train_batch"],
                test_batch_size=config["test_batch"],
                save_dir=config["results_dir"],
            )
        )


In [6]:
class SimpleLinear(torch.nn.Module):

    def __init__(self, h1=2048):
        super().__init__()
        self.fc1 = torch.nn.Linear(28*28, h1)
        self.fc2 = torch.nn.Linear(h1, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
# def train_fedavg_model(train_chunks, test_chunks, config):
# model = SimpleLinear()
# model.to(torch.device("cuda:0"))
# train_data = []
# train_labels = [chunk[1] for chunk in train_chunks]
# test_labels = [chunk[1] for chunk in test_chunks]
# test_data = []
# for i in range(config["num_clusters"]):
#     for j in range(config['num_clients_per_cluster']):
#         idx = i*config["num_clients_per_cluster"] + j
#         train_data.append()
# train_set = ClientDataset(data)

In [7]:
def calc_acc(model, device, client_data, train):
    loader = client_data.trainloader if train else client_data.testloader
    model.eval()
    acc = 0
    with torch.no_grad():
        for (X,Y) in loader:
            X = X.to(device)
            pred = model(X).argmax(axis=1).detach().cpu()
            acc += (Y == pred).float().mean()
    acc = acc/len(loader)
    acc *= 100.0
    return acc


class BaseTrainer(ABC):
    def __init__(self,config, save_dir):
        super(BaseTrainer, self).__init__()
        self.model = MODEL_LIST[config["model"]](**config["model_params"])
        self.save_dir = save_dir
        self.device = config["device"]
        self.loss_func = LOSSES[config["loss_func"]]
        self.config = config
        os.makedirs(self.save_dir, exist_ok=True)

    def train(self):
        raise NotImplementedError
    
    def test(self):
        raise NotImplementedError

    def load_model_weights(self):
        model_path  = os.path.join(self.save_dir, "model.pth")
        if os.path.exists(model_path):
            self.model.load_state_dict(torch.load(model_path))
        else:
            print("No model present at path : {}".format())

    def save_model_weights(self):
        model_path  = os.path.join(self.save_dir, "model.pth")
        torch.save(self.model.state_dict(), model_path)
    def save_metrics(self, train_loss, test_acc):
        torch.save({"train_loss": train_loss,  "test_acc" : test_acc}, os.path.join(self.save_dir,"metrics.pkl"))

class ClientTrainer(BaseTrainer):
    def __init__(self,  config, save_dir,client_id):
        super(ClientTrainer, self).__init__(config, save_dir)
        self.client_id = client_id
    
    def train(self, client_data):
        train_loss_list = []
        test_acc_list = []
        self.model.to(self.device)
        self.model.train()
        optimizer = OPTIMIZER_LIST[self.config["optimizer"]](self.model.parameters(), **self.config["optimizer_params"])
        for iteration in tqdm(range(self.config["iterations"])):
            self.model.zero_grad()
            (X,Y) = client_data.sample_batch(train=True)
            X = X.to(self.device)
            Y = Y.to(self.device)
            out = self.model(X)
            loss = self.loss_func(out, Y)
            loss.backward()
            optimizer.step()
            train_loss = loss.detach().cpu().numpy().item()
            train_loss_list.append(train_loss)
            test_acc = calc_acc(self.model, self.device, client_data, train=False)
            test_acc_list.append(test_acc)
            self.model.train()
            if iteration % self.config["save_freq"] == 0 or iteration == self.config["iterations"] - 1:
                self.save_model_weights()
                self.save_metrics(train_loss_list, test_acc_list)
            if iteration % self.config["print_freq"] == 0 or iteration == self.config["iterations"] - 1:
                print("Iteration : {} \n , Train Loss : {} \n, Test Acc : {} \n".format(iteration,  train_loss, test_acc))
                
        self.model.eval()
        self.model.cpu()


    def test(self, client_data):
        self.load_model_weights()
        self.model.eval()
        self.model.to(self.device)
        acc =  calc_acc(self.model, client_data)
        self.model.cpu()
        return acc


  
MODEL_LIST = {"lin" : SimpleLinear}
OPTIMIZER_LIST = {"sgd": optim.SGD, "adam": optim.Adam}
LOSSES = {"cross_entropy": nn.CrossEntropyLoss()}
# config["save_dir"] = os.path.join("./results")
config["iterations"] = 50
config["optimizer_params"] = {"lr":0.001}
config["save_freq"] = 2
config["print_freq"]  = 10
config["model"] = "lin"
config["optimizer"] = "adam"
config["loss_func"] = "cross_entropy"
#config["model_params"] = {"num_channels": 1 , "num_classes"  : 62}
config["model_params"] = {}
config["device"] = torch.device("cuda:0")
import pickle
client_trainers = [ClientTrainer(config,os.path.join(config["results_dir"], "init", "node_{}".format(i)), i) for i in range(config["num_clients"])]


for i in tqdm(range(config["num_clients"])):
    client_trainers[i].train(client_loaders[i])
    


  0%|          | 0/32 [00:00<?, ?it/s]

Iteration : 0 
 , Train Loss : 23.324874877929688 
, Test Acc : 33.73548126220703 



Iteration : 10 
 , Train Loss : 8.788775444030762 
, Test Acc : 76.26693725585938 



Iteration : 20 
 , Train Loss : 3.646134614944458 
, Test Acc : 87.07308197021484 



Iteration : 30 
 , Train Loss : 1.7003434896469116 
, Test Acc : 91.43390655517578 



...

Iteration : 20 
 , Train Loss : 4.280537128448486 
, Test Acc : 85.22884368896484 



Iteration : 30 
 , Train Loss : 1.8137036561965942 
, Test Acc : 89.28143310546875 



Iteration : 40 
 , Train Loss : 0.9283969402313232 
, Test Acc : 90.22746276855469 



100%|██████████| 32/32 [08:56<00:00, 16.75s/it]

Iteration : 49 
 , Train Loss : 0.4980047941207886 
, Test Acc : 90.9954605102539 



In [163]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(range(config["num_clients"]))
import itertools
def model_weights_diff(w_1, w_2):
    norm_sq = 0
    assert w_1.keys() == w_2.keys(), "Model weights have different keys"
    for key in w_1.keys():
        norm_sq  += (w_1[key] - w_2[key]).norm()**2
    return np.sqrt(norm_sq)
wt = client_trainers[0].model.state_dict()
# thresh = 0
# for key in wt.keys():
#     thresh += wt[key].norm()**2
# print(torch.sqrt(thresh))
# thresh = 37.68

all_pairs = list(itertools.combinations(range(config["num_clients"]),2))
arr = []
for pair in all_pairs:
    w_1  = client_trainers[pair[0]].model.state_dict()
    w_2 = client_trainers[pair[1]].model.state_dict()
    norm_diff = model_weights_diff(w_1, w_2)
    arr.append(norm_diff)
#thresh = torch.mean(torch.tensor(arr))
thresh = arr[torch.tensor(arr).argsort()[int(0.36*len(arr))-1]]
for i in range(len(all_pairs)):
    if arr[i] < thresh:
        G.add_edge(all_pairs[i][0], all_pairs[i][1])
G = G.to_undirected()
#cliques = list(nx.algorithms.clique.enumerate_all_cliques(G))


In [106]:
arr[torch.tensor(arr).argsort()[int(0.4*len(arr))]]

tensor(38.1052)

In [164]:
for node in G[0]:
    print(node)

1
4
9
17
18
20
22
23
28


In [165]:
adj = nx.convert_matrix.to_numpy_array(G)

In [108]:
adj.sum(axis=1)

array([ 9., 31., 11.,  8., 14.,  2., 11., 10.,  4., 26., 10.,  3.,  2.,
        7., 10., 20.,  8., 26., 17., 14., 25., 14., 24., 25.,  9.,  1.,
       12.,  4., 22.,  5.,  6.,  4.])

In [166]:
clustering = []
def correlation_clustering(G):
    global clustering
    if len(G.nodes) == 0:
        return
    else:
        cluster = []
        new_cluster_pivot = random.sample(G.nodes,1)[0]
        cluster.append(new_cluster_pivot)
        neighbors = G[new_cluster_pivot].copy()
        for node in neighbors:
            cluster.append(node)
            G.remove_node(node)
        G.remove_node(new_cluster_pivot)
        clustering.append(cluster)
        correlation_clustering(G)
correlation_clustering(G)

/tmp/ipykernel_27029/996340054.py:8: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  new_cluster_pivot = random.sample(G.nodes,1)[0]


In [167]:
clustering

[[24, 1, 9, 15, 18, 20, 23, 28],
 [26, 6, 17, 21, 22],
 [25],
 [30],
 [11],
 [0, 4],
 [19, 2, 7, 14, 29],
 [5],
 [13],
 [31],
 [27],
 [10],
 [3],
 [12],
 [16],
 [8]]

In [169]:
#config["t"] = 7
#t = config["t"]
clusters = [cluster  for cluster in clustering]
cluster_map = {i: clusters[i] for i in range(len(clusters))}
beta = 0.3

In [174]:
class ClusterTrainer(BaseTrainer):
    def __init__(self,  config, save_dir,cluster_id):
        super(ClusterTrainer, self).__init__(config, save_dir)
        self.cluster_id = cluster_id
    
    def train(self, client_data_list):
        num_clients = len(client_data_list)

        train_loss_list = []
        test_acc_list = []
        self.model.to(self.device)
        self.model.train()
        optimizer = OPTIMIZER_LIST[self.config["optimizer"]](self.model.parameters(), **self.config["optimizer_params"])
        for iteration in tqdm(range(self.config["iterations"])):
            trmean_buffer = {}
            for idx, param in self.model.named_parameters():
                trmean_buffer[idx] = []
            train_loss = 0
            for client in client_data_list:
                optimizer.zero_grad()
                (X,Y) = client.sample_batch()
                X = X.to(config["device"])
                Y = Y.to(config["device"])
                loss_func = nn.CrossEntropyLoss()
                out = self.model(X)
                loss = loss_func(out,Y)
                loss.backward()
                train_loss += loss.detach().cpu().numpy().item()
                with torch.no_grad():
                    for idx, param in self.model.named_parameters():
                        trmean_buffer[idx].append(param.grad.clone())
            train_loss = train_loss/num_clients
            optimizer.zero_grad()
            for idx, param in self.model.named_parameters():
                sorted, _  = torch.sort(torch.stack(trmean_buffer[idx], dim=0), dim=0)
                new_grad = sorted[int(beta*num_clients):int((1-beta)*num_clients),...].mean(dim=0)
                param.grad = new_grad
                trmean_buffer[idx] = []
            optimizer.step()
            
            train_loss_list.append(train_loss)
            test_acc = 0
            for client_data in client_data_list:
                test_acc += calc_acc(self.model, self.device, client_data, train=False)
            test_acc = test_acc/num_clients
            test_acc_list.append(test_acc)
            self.model.train()
            if iteration % self.config["save_freq"] == 0 or iteration == self.config["iterations"] - 1:
                self.save_model_weights()
                self.save_metrics(train_loss_list, test_acc_list)
            if iteration % self.config["print_freq"] == 0:
                print("Iteration : {} \n , Train Loss : {} \n, Test Acc : {} \n".format(iteration,  train_loss, test_acc))
                
        self.model.eval()
        self.model.cpu()


    def test(self, client_data_list):
        self.load_model_weights()
        self.model.eval()
        self.model.to(self.device)
        test_acc = 0
        for client_data in client_data_list:
            test_acc += calc_acc(self.model, self.device, client_data, train=False)
        self.model.cpu()
        return test_acc


In [175]:
%%time 
config["refine_steps"] = 3
for refine_step in tqdm(range(config["refine_steps"])):
    beta = 0.3
    cluster_trainers = []
    for cluster_id in tqdm(cluster_map.keys()):
        cluster_clients = [client_loaders[i] for i in cluster_map[cluster_id]]
        cluster_trainer = ClusterTrainer(config, os.path.join(config['results_dir'], "refine_{}".format(refine_step), "cluster_{}".format(cluster_id)), cluster_id)
        cluster_trainer.train(cluster_clients)
        cluster_trainers.append(cluster_trainer)
    with open(os.path.join(config["results_dir"],"refine_{}".format(refine_step), "cluster_maps.pkl"), 'wb') as handle:
        pickle.dump(cluster_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
    cluster_map_recluster = {}
    for key in cluster_map.keys():
        cluster_map_recluster[key] = []

    for i in tqdm(range(config["num_clients"])):
        w_node = client_trainers[i].model.state_dict()
        norm_diff = np.infty
        new_cluster_id = 0
        for cluster_id in cluster_map.keys():
            w_cluster = cluster_trainers[cluster_id].model.state_dict()
            curr_norm_diff = model_weights_diff(w_node, w_cluster)
            if norm_diff > curr_norm_diff:
                new_cluster_id = cluster_id
                norm_diff = curr_norm_diff
        
        cluster_map_recluster[new_cluster_id].append(i)
    cluster_map = cluster_map_recluster


    G = nx.Graph()
    G.add_nodes_from(cluster_map.keys())

    all_pairs = list(itertools.combinations(cluster_map.keys(),2))
    for pair in tqdm(all_pairs):
        w_1  = cluster_trainers[pair[0]].model.state_dict()
        w_2 = cluster_trainers[pair[1]].model.state_dict()
        norm_diff = model_weights_diff(w_1, w_2)
        if norm_diff < thresh:
            G.add_edge(pair[0], pair[1])
    G = G.to_undirected()
    clustering = []        
    correlation_clustering(G)
    merge_clusters = [cluster  for cluster in clustering]
    #merge_cluster_map = {i: clusters[i] for i in range(len(clusters))}
    #clusters = list(nx.algorithms.clique.enumerate_all_cliques(G))
    cluster_map_new = {}
    for i in range(len(clusters)):
        cluster_map_new[i] = []
        for j in merge_clusters[i]:
            cluster_map_new[i] += cluster_map[j]
    cluster_map = cluster_map_new


  0%|          | 0/3 [00:00<?, ?it/s]



Iteration : 0 
 , Train Loss : 25.816818237304688 
, Test Acc : 18.80019760131836 



Iteration : 10 
 , Train Loss : 32.171459436416626 
, Test Acc : 44.74650573730469 



In [83]:
with open(os.path.join(config["results_dir"],"refine_{}".format(refine_step), "cluster_maps.pkl"), 'wb') as handle:
    pickle.dump(cluster_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
cluster_map_recluster = {}
for key in cluster_map.keys():
    cluster_map_recluster[key] = []

for i in tqdm(range(config["num_clients"])):
    w_node = client_trainers[i].model.state_dict()
    norm_diff = np.infty
    new_cluster_id = 0
    for cluster_id in cluster_map.keys():
        w_cluster = cluster_trainers[cluster_id].model.state_dict()
        curr_norm_diff = model_weights_diff(w_node, w_cluster)
        if norm_diff > curr_norm_diff:
            new_cluster_id = cluster_id
            norm_diff = curr_norm_diff
    
    cluster_map_recluster[new_cluster_id].append(i)
cluster_map = cluster_map_recluster


G = nx.Graph()
G.add_nodes_from(cluster_map.keys())

all_pairs = list(itertools.combinations(cluster_map.keys(),2))
for pair in tqdm(all_pairs):
    w_1  = cluster_trainers[pair[0]].model.state_dict()
    w_2 = cluster_trainers[pair[1]].model.state_dict()
    norm_diff = model_weights_diff(w_1, w_2)
    if norm_diff < thresh:
        G.add_edge(pair[0], pair[1])
        
clusters = list(nx.algorithms.clique.enumerate_all_cliques(G))


100%|██████████| 12246/12246 [00:14<00:00, 820.48it/s]


In [85]:
cluster_map_new = {}
for i in range(len(clusters)):
    cluster_map_new[i] = []
    for j in clusters[i]:
        cluster_map_new[i] += cluster_map[j]
cluster_map = cluster_map_new

In [88]:
len(clusters)

668